项目目的：根据以往的账户消费情况，来预测下一个周期预算投入量

In [36]:
import numpy as np
import math 
history_data=[724.57,746.62,778.27,800.8,827.75,871.1,912.37,954.28,995.01,1037.2]
n=len(history_data)
X0=np.array(history_data)  #将历史数据转换为数组
#进行级比检验,判断是否能使用模型预测
L=[X0[i-1]/X0[i] for i in range(1,n)]
lmin=math.exp(-(2/(n+1)))
lmax=math.exp(2/(n+1))
for l in L:
    if lmin <l<lmax:
        continue
    else:
        print('数据不可进行高精度GM(1,1)模型预测')  
#运用累加生成（AGO），得到生成数组x1
history_data_agg=[sum(history_data[0:i+1])for i in range(n)]
x1=np.array(history_data_agg)
#计算数据矩阵B和数据向量Y
B=np.zeros([n-1,2])
Y=np.zeros([n-1,1])
for i in range(0,n-1):
    B[i][0]=-0.5*(x1[i]+x1[i+1])
    B[i][1]=1
    Y[i][0]=X0[i+1]
print('矩阵B:',B,'\n')
print('数据向量:',Y,'\n')
#计算GM（1,1）微分方程参数a和u
A=np.zeros([2,1])
A=np.linalg.inv(B.T.dot(B)).dot(B.T).dot(Y)
a=A[0][0]
u=A[1][0]
print('发展系数a:',a,'\n')
print('灰作用量u:',u,'\n')
#建立灰度预测模型
XX0=np.zeros(n)
XX0[0]=X0[0]  
for i in range(1,n):
    XX0[i]=(X0[0]-u/a)*(1-math.exp(a))*math.exp(-a*(i))  
### 对模型进行精度检验
#进行级比偏差值检验
P=[abs((1-(1-0.5*a)/(1+0.5*a)*l)) for l in L]
print('级比偏差值:',P,'\n')  #对所有的P，绝对值小于0.1，则达到最好，小于0.2，则达到一般要求
#开始进行误差检验
e=[]  
for i in range(0,n):
    e.append(X0[i]-XX0[i])
print('绝对误差值:',e,'\n')

omg=[]
for i in range(0,n):
    omg.append(e[i]/X0[i])
print('相对残差值:',omg,'\n')  #omg[i]一般要求小于20%，最好是小于10%

#求出相对残差以及其均值
eomg=0
for i in range(0,n):
    eomg+=abs(e[i]/X0[i])
eavg=eomg/(n-1)
#计算精度值p
p=1-eavg
print('精度值p:',p,'\n')  #一般要求p大于80%，最好是大于90%

#进行值预测
m=1  #往后预测的个数
f=np.zeros(m)  
for i in range(0,m):
    f[i]=round((X0[0]-u/a)*(1-math.exp(a))*math.exp(-a*(i+n)),2)
print('下个月的预算',f,'\n')


矩阵B: [[-1.097880e+03  1.000000e+00]
 [-1.860325e+03  1.000000e+00]
 [-2.649860e+03  1.000000e+00]
 [-3.464135e+03  1.000000e+00]
 [-4.313560e+03  1.000000e+00]
 [-5.205295e+03  1.000000e+00]
 [-6.138620e+03  1.000000e+00]
 [-7.113265e+03  1.000000e+00]
 [-8.129370e+03  1.000000e+00]] 

数据向量: [[ 746.62]
 [ 778.27]
 [ 800.8 ]
 [ 827.75]
 [ 871.1 ]
 [ 912.37]
 [ 954.28]
 [ 995.01]
 [1037.2 ]] 

发展系数a: -0.041977495531960354 

灰作用量u: 693.9402817303178 

级比偏差值: [0.012078039948577146, 0.0004666166275877348, 0.013536744101253051, 0.008923290103882486, 0.009021007493564381, 0.004295913537730622, 0.002923576638846015, 0.000188020990471216, 0.0004564991014397446] 

绝对误差值: [0.0, 6.8457768733799185, 6.780909326058804, -3.76360660059197, -11.306060404791538, -3.927240514375171, -0.17054141842857007, 2.6179246082357395, 2.549210990861752, 2.191415345450423] 

相对残差值: [0.0, 0.009169024233719856, 0.008712798034176834, -0.004699808442297665, -0.013658786354323814, -0.004508369319682208, -0.00018692133501

In [37]:
P.insert(0,0)
print(P)

[0, 0.012078039948577146, 0.0004666166275877348, 0.013536744101253051, 0.008923290103882486, 0.009021007493564381, 0.004295913537730622, 0.002923576638846015, 0.000188020990471216, 0.0004564991014397446]


In [38]:
#将数据呈现在表格中
import pandas as pd
data={'预算':history_data,'绝对误差值':e,'相对残差值':omg,'级比偏差值':P}
df=pd.DataFrame(data)
print(df)

        预算      绝对误差值     相对残差值     级比偏差值
0   724.57   0.000000  0.000000  0.000000
1   746.62   6.845777  0.009169  0.012078
2   778.27   6.780909  0.008713  0.000467
3   800.80  -3.763607 -0.004700  0.013537
4   827.75 -11.306060 -0.013659  0.008923
5   871.10  -3.927241 -0.004508  0.009021
6   912.37  -0.170541 -0.000187  0.004296
7   954.28   2.617925  0.002743  0.002924
8   995.01   2.549211  0.002562  0.000188
9  1037.20   2.191415  0.002113  0.000456


结论：级比偏差值的绝对值均小于0.1，相对残差值均小于0.2，精度值p: 0.9946273475375813，大于90%，说明模型能高精度预测。
    则下个周期预算投入可为：1079.38